In [1]:
import os
import pandas as pd
import numpy as np
import sklearn
import statsmodels.api as sm
import patsy
from patsy import dmatrices
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [2]:
merged = pd.read_csv("merged_with_gender.tsv", delimiter="\t")
# filter out examples where we failed to infer gender
genders = ['male', 'female']
df_filtered = merged[merged['gender'].isin(genders)]
# some cleaning
df_filtered["knowledge"] = pd.to_numeric(df_filtered["knowledge"].str.strip("&nbsp;"), errors='coerce')
# staff often missing
df_filtered["staff"] = pd.to_numeric(df_filtered["staff"].str.strip("&nbsp;"), errors='coerce')
df_filtered.shape

/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(53401, 11)

In [3]:
df_filtered.head()

,Unnamed: 0,dr-id,gendel,gender,helpful,insurance,knowledge,punctual,rating,review-text,staff
0,0,1,NaN,male,5,Insurance:Unknown,5,5,1,A CARING DOCTOR WHO CARES ABOUT HIS PATIENTS. ...,5
1,1,1,NaN,male,5,Insurance:Medicare,5,5,1,We have used Dr. Carden all of his career and ...,5
2,2,1,NaN,male,5,Insurance:Blue Cross / Blue Shield,5,5,1,"Very good doctor, caring. I think he knows e...",5
3,3,1,NaN,male,5,Insurance:United Healthcare,5,4,1,Dr. Carden is an outstanding physician. He spe...,4
4,4,1,NaN,male,5,Insurance:Unknown,5,5,1,Dr.Carden Is a great doctor that listens to hi...,5


In [4]:
grouped = df_filtered.groupby('dr-id')

Stolen from stanford people: https://raw.githubusercontent.com/stanfordnlp/CoreNLP/master/data/edu/stanford/nlp/patterns/surface/stopwords.txt

In [5]:
stop_words = [w.strip() for w in open("stopwords.txt").readlines()]
stop_words[:10]

['!!', '?!', '??', '!?', '`', '``', "''", '-lrb-', '-rrb-', '-lsb-']

Let's remove gender pronouns since these aren't terribly interesting as predictors

In [6]:
gender_pronouns = ["she", "her", "hers", "his", "he", "guy", "she's", "he's", "commentshe", "commentsshe", "man", "woman"]
stop_words.extend(gender_pronouns)

Merge reviews for individual doctors (we have multiple reviews for each!)

In [10]:
unique_drs = df_filtered["dr-id"].unique()
unique_drs.shape
grouped = df_filtered.groupby('dr-id')
texts, sexes, helpful, knowledge, punctual, staff, ids, lens = [], [], [], [], [], [], [], []
for i, dr in grouped:
    cur_text = " ".join(dr['review-text'].values)
    texts.append(cur_text)
    cur_sex = dr["gender"].values[0]

    helpful.append(dr["helpful"].mean(skipna=True)) # assuming missing at random!
    knowledge.append(dr["knowledge"].mean(skipna=True))
    punctual.append(dr["punctual"].mean(skipna=True))
    staff.append(dr["staff"].mean(skipna=True))

    ids.append(dr["dr-id"].values[0])
    sexes.append(cur_sex)
    
    lens.append(len(dr["review-text"].values))

dr_frame = pd.DataFrame({"sex":sexes, "y_helpful":helpful, "y_knowledge":knowledge, "y_punctual":punctual, "y_staff":staff, "id":ids})

In [209]:
sum(lens)/float(len(lens))

3.1854075691411934

How many females v males?

In [285]:
sexes.count("female")/float(len(sexes))
sexes.count("female")

4662

So about 72% male

Note; should probably pull out and pool reviews for physicians -- here we treat individual reviews for the same physician as multiple / independent instances

## Regression analysis for gender-score 

**Important caveats** we are making several assumptions here, including:
* We are treating the means as a point estimates and not explicitly weighting by variance
* We are ignoring what are likely confounders, including specialties -- e.g., there may be certain specialties that receive lower scores and perhaps women are overrepresented in these. We have no way of knowing.

In [286]:
y_helpful, X = dmatrices('y_helpful ~ sex', data=dr_frame.dropna(subset=["y_helpful"]), return_type='dataframe')
mod = sm.OLS(y_helpful, X)
res_helpful = mod.fit()
print(res_helpful.summary())

                            OLS Regression Results                            
Dep. Variable:              y_helpful   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     13.99
Date:                Sat, 29 Oct 2016   Prob (F-statistic):           0.000185
Time:                        09:43:59   Log-Likelihood:                -28355.
No. Observations:               16485   AIC:                         5.671e+04
Df Residuals:                   16483   BIC:                         5.673e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept       3.6467      0.020    184.220      

In [301]:
import coef_plots
import importlib
importlib.reload(coef_plots)
point_estimates = np.copy(res_helpful.params.values)
# add intercept to male factor
point_estimates[1] = point_estimates[0]+point_estimates[1]
lowers = point_estimates - 1.96*res_helpful.bse.values
uppers = point_estimates + 1.96*res_helpful.bse.values
names = ["female", "male"]
colors = ["black", "blue"]
coef_plots.point_ests_and_CIs(point_estimates, lowers, uppers, names, colors, "helpfulness.pdf", xtitle="Helpfulness" )

/anaconda/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [302]:
res_helpful.params.values

array([ 3.64665305,  0.0874156 ])

In [303]:
print(res_helpful.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}    &    y_helpful     & \textbf{  R-squared:         } &     0.001   \\
\textbf{Model:}            &       OLS        & \textbf{  Adj. R-squared:    } &     0.001   \\
\textbf{Method:}           &  Least Squares   & \textbf{  F-statistic:       } &     13.99   \\
\textbf{Date:}             & Sat, 29 Oct 2016 & \textbf{  Prob (F-statistic):} &  0.000185   \\
\textbf{Time:}             &     09:55:56     & \textbf{  Log-Likelihood:    } &   -28355.   \\
\textbf{No. Observations:} &       16485      & \textbf{  AIC:               } & 5.671e+04   \\
\textbf{Df Residuals:}     &       16483      & \textbf{  BIC:               } & 5.673e+04   \\
\textbf{Df Model:}         &           1      & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tabular}{lccccc}
                     & \textbf{coef} & \textbf{std err} & \textbf{t} & \textbf{P$>$$|$t$|$} & \textbf{[95.0\% Conf. Int.]}  \\
\midrule


In [289]:
y_knowledge, X = dmatrices('y_knowledge ~ sex', data=dr_frame.dropna(subset=["y_knowledge"]), return_type='dataframe')
mod = sm.OLS(y_knowledge, X)
res_knowledge = mod.fit()
print(res_knowledge.summary())

                            OLS Regression Results                            
Dep. Variable:            y_knowledge   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     17.44
Date:                Sat, 29 Oct 2016   Prob (F-statistic):           2.98e-05
Time:                        09:43:59   Log-Likelihood:                -27077.
No. Observations:               16485   AIC:                         5.416e+04
Df Residuals:                   16483   BIC:                         5.417e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept       3.8143      0.018    208.224      

In [300]:
print(res_knowledge.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}    &   y_knowledge    & \textbf{  R-squared:         } &     0.001   \\
\textbf{Model:}            &       OLS        & \textbf{  Adj. R-squared:    } &     0.001   \\
\textbf{Method:}           &  Least Squares   & \textbf{  F-statistic:       } &     17.44   \\
\textbf{Date:}             & Sat, 29 Oct 2016 & \textbf{  Prob (F-statistic):} &  2.98e-05   \\
\textbf{Time:}             &     09:48:40     & \textbf{  Log-Likelihood:    } &   -27077.   \\
\textbf{No. Observations:} &       16485      & \textbf{  AIC:               } & 5.416e+04   \\
\textbf{Df Residuals:}     &       16483      & \textbf{  BIC:               } & 5.417e+04   \\
\textbf{Df Model:}         &           1      & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tabular}{lccccc}
                     & \textbf{coef} & \textbf{std err} & \textbf{t} & \textbf{P$>$$|$t$|$} & \textbf{[95.0\% Conf. Int.]}  \\
\midrule


In [304]:
import coef_plots
import importlib
importlib.reload(coef_plots)
point_estimates = np.copy(res_knowledge.params.values)
# add intercept to male factor
point_estimates[1] = point_estimates[0]+point_estimates[1]
lowers = point_estimates - 1.96*res_knowledge.bse.values
uppers = point_estimates + 1.96*res_knowledge.bse.values
names = ["female", "male"]
colors = ["black", "blue"]
coef_plots.point_ests_and_CIs(point_estimates, lowers, uppers, names, colors, "knowledge.pdf", xtitle="Knowledge", turn_off_y=True )

/anaconda/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [305]:
y_punctual, X = dmatrices('y_punctual ~ sex', data=dr_frame.dropna(subset=["y_punctual"]), return_type='dataframe')
mod = sm.OLS(y_punctual, X)
res_punc = mod.fit()
print(res_punc.summary())

                            OLS Regression Results                            
Dep. Variable:             y_punctual   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     41.23
Date:                Sat, 29 Oct 2016   Prob (F-statistic):           1.39e-10
Time:                        10:02:03   Log-Likelihood:                -26066.
No. Observations:               16469   AIC:                         5.214e+04
Df Residuals:                   16467   BIC:                         5.215e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept       3.6203      0.017    209.700      

In [307]:
print(res_punc.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}    &    y_punctual    & \textbf{  R-squared:         } &     0.002   \\
\textbf{Model:}            &       OLS        & \textbf{  Adj. R-squared:    } &     0.002   \\
\textbf{Method:}           &  Least Squares   & \textbf{  F-statistic:       } &     41.23   \\
\textbf{Date:}             & Sat, 29 Oct 2016 & \textbf{  Prob (F-statistic):} &  1.39e-10   \\
\textbf{Time:}             &     10:02:20     & \textbf{  Log-Likelihood:    } &   -26066.   \\
\textbf{No. Observations:} &       16469      & \textbf{  AIC:               } & 5.214e+04   \\
\textbf{Df Residuals:}     &       16467      & \textbf{  BIC:               } & 5.215e+04   \\
\textbf{Df Model:}         &           1      & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tabular}{lccccc}
                     & \textbf{coef} & \textbf{std err} & \textbf{t} & \textbf{P$>$$|$t$|$} & \textbf{[95.0\% Conf. Int.]}  \\
\midrule


In [308]:
import coef_plots
import importlib
importlib.reload(coef_plots)
point_estimates = np.copy(res_punc.params.values)
# add intercept to male factor
point_estimates[1] = point_estimates[0]+point_estimates[1]
lowers = point_estimates - 1.96*res_punc.bse.values
uppers = point_estimates + 1.96*res_punc.bse.values
names = ["female", "male"]
colors = ["black", "blue"]
coef_plots.point_ests_and_CIs(point_estimates, lowers, uppers, names, colors, "punctual.pdf", xtitle="Punctuality", turn_off_y=True )

/anaconda/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [309]:
y_staff, X = dmatrices('y_staff ~ sex', data=dr_frame.dropna(subset=["y_staff"]), return_type='dataframe')
mod = sm.OLS(y_staff, X)
res_staff = mod.fit()
print(res_staff.summary())

                            OLS Regression Results                            
Dep. Variable:                y_staff   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     29.07
Date:                Sat, 29 Oct 2016   Prob (F-statistic):           7.10e-08
Time:                        10:04:53   Log-Likelihood:                -24477.
No. Observations:               15290   AIC:                         4.896e+04
Df Residuals:                   15288   BIC:                         4.897e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept       3.7263      0.018    204.292      

In [310]:
print(res_staff.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}    &     y_staff      & \textbf{  R-squared:         } &     0.002   \\
\textbf{Model:}            &       OLS        & \textbf{  Adj. R-squared:    } &     0.002   \\
\textbf{Method:}           &  Least Squares   & \textbf{  F-statistic:       } &     29.07   \\
\textbf{Date:}             & Sat, 29 Oct 2016 & \textbf{  Prob (F-statistic):} &  7.10e-08   \\
\textbf{Time:}             &     10:05:18     & \textbf{  Log-Likelihood:    } &   -24477.   \\
\textbf{No. Observations:} &       15290      & \textbf{  AIC:               } & 4.896e+04   \\
\textbf{Df Residuals:}     &       15288      & \textbf{  BIC:               } & 4.897e+04   \\
\textbf{Df Model:}         &           1      & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tabular}{lccccc}
                     & \textbf{coef} & \textbf{std err} & \textbf{t} & \textbf{P$>$$|$t$|$} & \textbf{[95.0\% Conf. Int.]}  \\
\midrule


In [294]:
import coef_plots
import importlib
importlib.reload(coef_plots)
point_estimates = np.copy(res_staff.params.values)
# add intercept to male factor
point_estimates[1] = point_estimates[0]+point_estimates[1]
lowers = point_estimates - 1.96*res_staff.bse.values
uppers = point_estimates + 1.96*res_staff.bse.values
names = ["female", "male"]
colors = ["black", "blue"]
coef_plots.point_ests_and_CIs(point_estimates, lowers, uppers, names, colors, "staff.pdf", xtitle="Staff" )

/anaconda/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


## Predictive analysis; I think Michael's is much better!

In [295]:
vectorizer = CountVectorizer(max_features=20000, min_df=25, ngram_range=(1,1), binary=True, stop_words=stop_words)
X = vectorizer.fit_transform(texts)
y = sexes
param_grid = {"C":[.01, .1, 1]}
clf = GridSearchCV(LogisticRegression(), param_grid=param_grid, scoring="f1_macro")

In [296]:
clf.fit(X,y)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={'C': [0.01, 0.1, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1_macro', verbose=0)

In [297]:
clf.best_score_

0.54194365591947247

In [298]:
def show_most_informative_features(vectorizer, clf, n=50):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [299]:
show_most_informative_features(vectorizer, clf.best_estimator_, n=100)

	-1.7046	seemingly      		2.0930	vague          
	-1.6818	inpatient      		2.0733	abruptly       
	-1.5260	yeast          		1.8747	cocky          
	-1.4709	reference      		1.7473	jerk           
	-1.4381	mary           		1.6032	pursue         
	-1.4349	sweetest       		1.5596	released       
	-1.4067	partum         		1.5320	prostate       
	-1.3976	eyelid         		1.5188	bleed          
	-1.3944	gynecological  		1.5130	covering       
	-1.3731	gown           		1.4894	sensitivity    
	-1.3725	ultra          		1.4754	introduce      
	-1.3722	judgmental     		1.4023	endoscopy      
	-1.3138	sooo           		1.3919	numb           
	-1.3039	assess         		1.3835	relative       
	-1.2656	communicative  		1.3684	delaware       
	-1.2651	sounded        		1.3524	load           
	-1.2547	suicide        		1.3518	playing        
	-1.2503	connected      		1.3170	needles        
	-1.2482	linda          		1.2973	miller         
	-1.2461	1500           		1.2698	canceled       
	-1.2455	argued     